In [ ]:
!pip install -q transformers accelerate peft bitsandbytes fastapi uvicorn pyngrok sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.6 MB/s eta 0:00:00


In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU name: Tesla T4


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

print("Loading base model...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully!")

Loading base model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
prompt = "Q: Who are you?\nA:"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Q: Who are you?
A: I'm Phi. I work for Microsoft.
Q: What can you do?
A: I'm a large language model trained by Microsoft. I can help with a wide range of tasks, such as answering questions, writing ess


In [ ]:
for name, module in model.named_modules():
    if "attn" in name.lower() and isinstance(module, torch.nn.Linear):
        print(name)

model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.3.self_attn.o_proj
model.layers.3.self_attn.qkv_proj
model.layers.4.self_attn.o_proj
model.layers.4.self_attn.qkv_proj
model.layers.5.self_attn.o_proj
model.layers.5.self_attn.qkv_proj
model.layers.6.self_attn.o_proj
model.layers.6.self_attn.qkv_proj
model.layers.7.self_attn.o_proj
model.layers.7.self_attn.qkv_proj
model.layers.8.self_attn.o_proj
model.layers.8.self_attn.qkv_proj
model.layers.9.self_attn.o_proj
model.layers.9.self_attn.qkv_proj
model.layers.10.self_attn.o_proj
model.layers.10.self_attn.qkv_proj
model.layers.11.self_attn.o_proj
model.layers.11.self_attn.qkv_proj
model.layers.12.self_attn.o_proj
model.layers.12.self_attn.qkv_proj
model.layers.13.self_attn.o_proj
model.layers.13.self_attn.qkv_proj
model.layers.14.self_attn.o_proj
model.layers.14.self_attn.qkv_proj


In [ ]:
from peft import LoraConfig, get_peft_model

target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]  # Confirmed from print output

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,145,728 || all params: 3,824,225,280 || trainable%: 0.0823


In [ ]:
train_data = [
    {'question': 'Which faculty specialize in Machine Learning in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.'},
    {'question': 'Faculty who did specialization in Machine Learning?',
     'answer': 'The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.'},
    {'question': 'List the CSE faculty who work in Machine Learning.',
     'answer': 'The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.'},
    {'question': 'Who are the experts in Machine Learning at SRKR?',
     'answer': 'The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.'},
    {'question': 'Name the SRKR CSE faculty working in Machine Learning.',
     'answer': 'The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.'},

    {'question': 'Which faculty specialized in Cloud Computing in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Cloud Computing are: Chintapalli Siva Subrahmanyam, B. Teja Sree, N. Deshai, G. Sandeep, Dr. S. Venkataramana.'},
    {'question': 'Faculty who did specialization in Cloud Computing?',
     'answer': 'The faculty specialized in Cloud Computing are: Chintapalli Siva Subrahmanyam, B. Teja Sree, N. Deshai, G. Sandeep, Dr. S. Venkataramana.'},
    {'question': 'List the faculty who work in the area of Cloud Computing in CSE at SRKR.',
     'answer': 'The faculty specialized in Cloud Computing are: Chintapalli Siva Subrahmanyam, B. Teja Sree, N. Deshai, G. Sandeep, Dr. S. Venkataramana.'},
    {'question': 'Who are the CSE faculty with expertise in Cloud Computing at SRKR?',
     'answer': 'The faculty specialized in Cloud Computing are: Chintapalli Siva Subrahmanyam, B. Teja Sree, N. Deshai, G. Sandeep, Dr. S. Venkataramana.'},
    {'question': 'Name the CSE faculty working in Cloud Computing?',
     'answer': 'The faculty specialized in Cloud Computing are: Chintapalli Siva Subrahmanyam, B. Teja Sree, N. Deshai, G. Sandeep, Dr. S. Venkataramana.'},

    {'question': 'Which faculty specialized in Data Mining in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Data Mining are: S. Suryanarayanaraju, Dr. I. Hemalatha, K. Rajeswari, B. Teja Sree, V K S K Sai Vadapalli, Dr. K Kishore Raju, N. Deshai, B. Lakshmi, Dr. S. Rama Gopala Reddy, Ch. Navya.'},
    {'question': 'Faculty who did specialization in Data Mining?',
     'answer': 'The faculty specialized in Data Mining are: S. Suryanarayanaraju, Dr. I. Hemalatha, K. Rajeswari, B. Teja Sree, V K S K Sai Vadapalli, Dr. K Kishore Raju, N. Deshai, B. Lakshmi, Dr. S. Rama Gopala Reddy, Ch. Navya.'},
    {'question': 'List the faculty who work in the area of Data Mining in CSE at SRKR.',
     'answer': 'The faculty specialized in Data Mining are: S. Suryanarayanaraju, Dr. I. Hemalatha, K. Rajeswari, B. Teja Sree, V K S K Sai Vadapalli, Dr. K Kishore Raju, N. Deshai, B. Lakshmi, Dr. S. Rama Gopala Reddy, Ch. Navya.'},
    {'question': 'Who are the CSE faculty with expertise in Data Mining at SRKR?',
     'answer': 'The faculty specialized in Data Mining are: S. Suryanarayanaraju, Dr. I. Hemalatha, K. Rajeswari, B. Teja Sree, V K S K Sai Vadapalli, Dr. K Kishore Raju, N. Deshai, B. Lakshmi, Dr. S. Rama Gopala Reddy, Ch. Navya.'},
    {'question': 'Name the CSE faculty working in Data Mining?',
     'answer': 'The faculty specialized in Data Mining are: S. Suryanarayanaraju, Dr. I. Hemalatha, K. Rajeswari, B. Teja Sree, V K S K Sai Vadapalli, Dr. K Kishore Raju, N. Deshai, B. Lakshmi, Dr. S. Rama Gopala Reddy, Ch. Navya.'},

    {'question': 'Which faculty specialized in Big Data in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Big Data are: S. Suryanarayanaraju, Dr. I. Hemalatha, Dr. K Kishore Raju, N. Deshai, Ch. Navya.'},
    {'question': 'Faculty who did specialization in Big Data?',
     'answer': 'The faculty specialized in Big Data are: S. Suryanarayanaraju, Dr. I. Hemalatha, Dr. K Kishore Raju, N. Deshai, Ch. Navya.'},
    {'question': 'List the faculty who work in the area of Big Data in CSE at SRKR.',
     'answer': 'The faculty specialized in Big Data are: S. Suryanarayanaraju, Dr. I. Hemalatha, Dr. K Kishore Raju, N. Deshai, Ch. Navya.'},
    {'question': 'Who are the CSE faculty with expertise in Big Data at SRKR?',
     'answer': 'The faculty specialized in Big Data are: S. Suryanarayanaraju, Dr. I. Hemalatha, Dr. K Kishore Raju, N. Deshai, Ch. Navya.'},
    {'question': 'Name the CSE faculty working in Big Data?',
     'answer': 'The faculty specialized in Big Data are: S. Suryanarayanaraju, Dr. I. Hemalatha, Dr. K Kishore Raju, N. Deshai, Ch. Navya.'},

    {'question': 'Which faculty specialized in Image Processing in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Image Processing are: M. Naga Sai, Dr. I. Hemalatha, Dr. B V D S Sekhar, Kadali Srinivas, Dr. M Krishna Satya Varma, P. Hema Sri.'},
    {'question': 'Faculty who did specialization in Image Processing?',
     'answer': 'The faculty specialized in Image Processing are: M. Naga Sai, Dr. I. Hemalatha, Dr. B V D S Sekhar, Kadali Srinivas, Dr. M Krishna Satya Varma, P. Hema Sri.'},
    {'question': 'List the faculty who work in the area of Image Processing in CSE at SRKR.',
     'answer': 'The faculty specialized in Image Processing are: M. Naga Sai, Dr. I. Hemalatha, Dr. B V D S Sekhar, Kadali Srinivas, Dr. M Krishna Satya Varma, P. Hema Sri.'},
    {'question': 'Who are the CSE faculty with expertise in Image Processing at SRKR?',
     'answer': 'The faculty specialized in Image Processing are: M. Naga Sai, Dr. I. Hemalatha, Dr. B V D S Sekhar, Kadali Srinivas, Dr. M Krishna Satya Varma, P. Hema Sri.'},
    {'question': 'Name the CSE faculty working in Image Processing?',
     'answer': 'The faculty specialized in Image Processing are: M. Naga Sai, Dr. I. Hemalatha, Dr. B V D S Sekhar, Kadali Srinivas, Dr. M Krishna Satya Varma, P. Hema Sri.'},

    {'question': 'Which faculty specialized in Cybersecurity in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Cybersecurity are: R. Mounika, Dr. Ravi Kiran Varma P B E, V. Ramesh, R. Anusha, Galeiah M, A. Suma Lakshmi.'},
    {'question': 'Faculty who did specialization in Cybersecurity?',
     'answer': 'The faculty specialized in Cybersecurity are: R. Mounika, Dr. Ravi Kiran Varma P B E, V. Ramesh, R. Anusha, Galeiah M, A. Suma Lakshmi.'},
    {'question': 'List the faculty who work in the area of Cybersecurity in CSE at SRKR.',
     'answer': 'The faculty specialized in Cybersecurity are: R. Mounika, Dr. Ravi Kiran Varma P B E, V. Ramesh, R. Anusha, Galeiah M, A. Suma Lakshmi.'},
    {'question': 'Who are the CSE faculty with expertise in Cybersecurity at SRKR?',
     'answer': 'The faculty specialized in Cybersecurity are: R. Mounika, Dr. Ravi Kiran Varma P B E, V. Ramesh, R. Anusha, Galeiah M, A. Suma Lakshmi.'},
    {'question': 'Name the CSE faculty working in Cybersecurity?',
     'answer': 'The faculty specialized in Cybersecurity are: R. Mounika, Dr. Ravi Kiran Varma P B E, V. Ramesh, R. Anusha, Galeiah M, A. Suma Lakshmi.'},

    {'question': 'Which faculty specialized in IoT in the CSE department at SRKR?',
     'answer': 'The faculty specialized in IoT are: B. Teja Sree, Dr. JMSV Ravi Kumar, P. Chandrasekhar, Dr. Ravi Kiran Varma P B E, N. Deshai, V. Ramesh, D. Sai Kumar, CH. Dileep Chakravarthy.'},
    {'question': 'Faculty who did specialization in IoT?',
     'answer': 'The faculty specialized in IoT are: B. Teja Sree, Dr. JMSV Ravi Kumar, P. Chandrasekhar, Dr. Ravi Kiran Varma P B E, N. Deshai, V. Ramesh, D. Sai Kumar, CH. Dileep Chakravarthy.'},
    {'question': 'List the faculty who work in the area of IoT in CSE at SRKR.',
     'answer': 'The faculty specialized in IoT are: B. Teja Sree, Dr. JMSV Ravi Kumar, P. Chandrasekhar, Dr. Ravi Kiran Varma P B E, N. Deshai, V. Ramesh, D. Sai Kumar, CH. Dileep Chakravarthy.'},
    {'question': 'Who are the CSE faculty with expertise in IoT at SRKR?',
     'answer': 'The faculty specialized in IoT are: B. Teja Sree, Dr. JMSV Ravi Kumar, P. Chandrasekhar, Dr. Ravi Kiran Varma P B E, N. Deshai, V. Ramesh, D. Sai Kumar, CH. Dileep Chakravarthy.'},
    {'question': 'Name the CSE faculty working in IoT?',
     'answer': 'The faculty specialized in IoT are: B. Teja Sree, Dr. JMSV Ravi Kumar, P. Chandrasekhar, Dr. Ravi Kiran Varma P B E, N. Deshai, V. Ramesh, D. Sai Kumar, CH. Dileep Chakravarthy.'},

    {'question': 'Which faculty specialized in Networks in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Networks are: Chintapalli Siva Subrahmanyam, Dr. P. Krishna, P. Chandrasekhar, Dr. Ch Diwakar, CH. Dileep Chakravarthy, C. Kusuma, A. Suma Lakshmi.'},
    {'question': 'Faculty who did specialization in Networks?',
     'answer': 'The faculty specialized in Networks are: Chintapalli Siva Subrahmanyam, Dr. P. Krishna, P. Chandrasekhar, Dr. Ch Diwakar, CH. Dileep Chakravarthy, C. Kusuma, A. Suma Lakshmi.'},
    {'question': 'List the faculty who work in the area of Networks in CSE at SRKR.',
     'answer': 'The faculty specialized in Networks are: Chintapalli Siva Subrahmanyam, Dr. P. Krishna, P. Chandrasekhar, Dr. Ch Diwakar, CH. Dileep Chakravarthy, C. Kusuma, A. Suma Lakshmi.'},
    {'question': 'Who are the CSE faculty with expertise in Networks at SRKR?',
     'answer': 'The faculty specialized in Networks are: Chintapalli Siva Subrahmanyam, Dr. P. Krishna, P. Chandrasekhar, Dr. Ch Diwakar, CH. Dileep Chakravarthy, C. Kusuma, A. Suma Lakshmi.'},
    {'question': 'Name the CSE faculty working in Networks?',
     'answer': 'The faculty specialized in Networks are: Chintapalli Siva Subrahmanyam, Dr. P. Krishna, P. Chandrasekhar, Dr. Ch Diwakar, CH. Dileep Chakravarthy, C. Kusuma, A. Suma Lakshmi.'},

    {'question': 'Which faculty specialized in Full Stack in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'Faculty who did specialization in Full Stack?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'List the faculty who work in the area of Full Stack in CSE at SRKR.',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'Who are the CSE faculty with expertise in Full Stack at SRKR?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'Name the CSE faculty working in Full Stack?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},

    {'question': 'Which faculty specialized in Full Stack in the CSE department at SRKR?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'Faculty who did specialization in Full Stack?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'List the faculty who work in the area of Full Stack in CSE at SRKR.',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'Who are the CSE faculty with expertise in Full Stack at SRKR?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},
    {'question': 'Name the CSE faculty working in Full Stack?',
     'answer': 'The faculty specialized in Full Stack are: B. Teja Sree.'},

    {"question": "What is the NAAC accreditation grade of SRKR Engineering College?",
     "answer": "SRKR Engineering College has an NAAC A+ grade with a score of 3.42 out of 4."},

    {"question": "What accreditation does SRKR have?",
     "answer": "SRKR is accredited with NAAC A+ grade, scoring 3.42 out of 4."},

    {"question": "SRKR NAAC rating?",
     "answer": "NAAC A+ with a CGPA of 3.42 out of 4."},

    {"question": "Is SRKR NAAC approved?",
     "answer": "Yes, SRKR has NAAC A+ accreditation."},

    {"question": "Who is the Principal of SRKR Engineering College?",
     "answer": "The Principal of SRKR Engineering College is Prof. K. V. Murali Krishnam Raju."},

    {"question": "Name the Principal of SRKR College?",
     "answer": "Prof. K. V. Murali Krishnam Raju is the Principal of SRKR."},

    {"question": "Who heads SRKR Engineering College?",
     "answer": "Prof. K. V. Murali Krishnam Raju is the head of SRKR as Principal."},

    {"question": "What is the annual fee for CSE at SRKR?",
     "answer": "The annual fee for CSE is ₹1,05,000."},

    {"question": "CSE fee at SRKR?",
     "answer": "₹1,05,000 per year for CSE."},

    {"question": "How much do CSE students pay annually at SRKR?",
     "answer": "They pay ₹1,05,000 per year for CSE."},

    {"question": "Fee structure for CSE?",
     "answer": "₹1,05,000 per year at SRKR Engineering College."},

    {"question": "What is the intake for CSE at SRKR?",
     "answer": "The CSE intake at SRKR Engineering College is 300 seats."},

    {"question": "How many CSE seats does SRKR have?",
     "answer": "300 seats for B.Tech in CSE."},

    {"question": "CSE admission capacity in SRKR?",
     "answer": "It offers 300 B.Tech seats in CSE."},

    {"question": "Which programs in SRKR are NBA accredited?",
     "answer": "SRKR has 6 NBA accredited programs including CSE, IT, ECE, EEE, Civil, and Mechanical."},

    {"question": "How many NBA programs does SRKR have?",
     "answer": "SRKR has 6 NBA accredited programs including CSE and IT."},

    {"question": "Who is the HOD of CSE at SRKR?",
     "answer": "Dr. Bh. V. S. R. K. Raju is the Head of the CSE department at SRKR Engineering College."},
    {"question": "Who and when srkr was established?",
     "answer": "Dr. sagi rama krishnam raju established SRKR Engineering College.in 1980"},

    {"question": "Name the HOD of CSE in SRKR Engineering College",
     "answer": "The HOD of the CSE department is Dr. Bh. V. S. R. K. Raju."},

    {"question": "Who heads the CSE department at SRKR?",
     "answer": "The CSE department is headed by Dr. Bh. V. S. R. K. Raju."},

    {"question": "CSE HOD at SRKR?",
     "answer": "Dr. Bh. V. S. R. K. Raju."},

    {"question": "Who is the CSE department in-charge in SRKR?",
     "answer": "The department in-charge of CSE is Dr. Bh. V. S. R. K. Raju."},

    {"question": "Who is the chief of CSE Department in SRKR Engineering College?",
     "answer": "The chief of the CSE Department is Dr. Bh. V. S. R. K. Raju."},

    {"question": "Who manages the CSE academics and administration in SRKR?",
     "answer": "Dr. Bh. V. S. R. K. Raju manages the CSE academics and administration as the Head of the Department."}

]
def format_example(example):
    return f"### Question: {example['question']}\n### Answer: {example['answer']}"

train_texts = [format_example(x) for x in train_data]
train_texts

['### Question: Which faculty specialize in Machine Learning in the CSE department at SRKR?\n### Answer: The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.',
 '### Question: Faculty who did specialization in Machine Learning?\n### Answer: The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.',
 '### Question: List the CSE faculty who work in Machine Learning.\n### Answer: The faculty specialized in Machine Learning are: Dr. K. Satyanarayana Raju, Dr. K Kishore Raju, PDSS Lakshmi Kumari, V K S K Sai Vadapalli, Dr. I. Hemalatha, K. Rajeswari, C. Kusuma, Galeiah M.',
 '### Question: Who are the experts in Machine Learning at SRKR?\n### Answer: The faculty specialized in Machine Learning are: Dr. K. Satyanarayan

In [ ]:
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        for txt in texts:
            enc = tokenizer(
                txt,
                return_tensors="pt",
                padding="max_length",
                max_length=256,
                truncation=True
            )
            self.input_ids.append(enc.input_ids.squeeze())
            self.attn_masks.append(enc.attention_mask.squeeze())

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attn_masks[idx],
            "labels": self.input_ids[idx]
        }

dataset = QADataset(train_texts, tokenizer)
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

print("Dataset ready with", len(dataset), "samples.")

Dataset ready with 74 samples.


In [ ]:
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from transformers import get_linear_schedule_with_warmup
import torch, gc
import numpy as np

model.train()

EPOCHS = 12
LEARNING_RATE = 1e-4
GRADIENT_CLIP = 1.0
WARMUP_STEPS = 10

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

num_training_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=num_training_steps
)

best_loss = float('inf')

for epoch in range(EPOCHS):
    epoch_losses = []

    for batch in train_dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        epoch_losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), GRADIENT_CLIP)

        optimizer.step()
        scheduler.step()
        torch.cuda.empty_cache()
        gc.collect()

    avg_loss = np.mean(epoch_losses)
    print(f"Epoch {epoch+1}/{EPOCHS} - Avg Loss: {avg_loss:.4f}")
    if avg_loss < best_loss:
        best_loss = avg_loss
        model.save_pretrained(f"best_model_checkpoint_epoch_{epoch+1}")
        print(" Saved Best Model Checkpoint!")

print("Training completed successfully!")


Epoch 1/12 - Avg Loss: 15.4130
 Saved Best Model Checkpoint!
Epoch 2/12 - Avg Loss: 15.0515
 Saved Best Model Checkpoint!
Epoch 3/12 - Avg Loss: 14.9456
 Saved Best Model Checkpoint!
Epoch 4/12 - Avg Loss: 14.9073
 Saved Best Model Checkpoint!
Epoch 5/12 - Avg Loss: 14.8906
 Saved Best Model Checkpoint!
Epoch 6/12 - Avg Loss: 14.8848
 Saved Best Model Checkpoint!
Epoch 7/12 - Avg Loss: 14.8807
 Saved Best Model Checkpoint!
Epoch 8/12 - Avg Loss: 14.8785
 Saved Best Model Checkpoint!
Epoch 9/12 - Avg Loss: 14.8772
 Saved Best Model Checkpoint!
Epoch 10/12 - Avg Loss: 14.8763
 Saved Best Model Checkpoint!
Epoch 11/12 - Avg Loss: 14.8755
 Saved Best Model Checkpoint!
Epoch 12/12 - Avg Loss: 14.8749
 Saved Best Model Checkpoint!
Training completed successfully!


In [ ]:
def ask_model(question):
    model.eval()
    prompt = f"### Question: {question}\n### Answer:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    response = text[len(prompt):]


    response = response.split("### Question:")[0].strip()

    return response
print("Fine-Tuned Response:")
print(ask_model("name some of the faculty of cyber security?"))

Fine-Tuned Response:
The faculty specialized in Cybersecurity are: R. Mounika, Dr. Ravi Kiran Varma P B E, V. Ramesh, R. Anusha, Galeiah M, A. Suma Lakshmi.


In [ ]:
adapter_path = "/content/drive/MyDrive/system_labs/adapters/srkr_expert"

model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

print("Adapter saved at:", adapter_path)


Adapter saved at: /content/drive/MyDrive/system_labs/adapters/srkr_expert


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
def cleanup_gpu():
    import torch, gc
    gc.collect()
    torch.cuda.empty_cache()
    print("GPU memory cleared!")

cleanup_gpu()



GPU memory cleared!


In [ ]:
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import tempfile

BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
ADAPTER_PATH = "/content/drive/MyDrive/system_labs/adapters/srkr_expert"

tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)

# Create a temporary directory for offloading
offload_folder = tempfile.mkdtemp()
print(f"Using offload folder: {offload_folder}")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=offload_folder  # Specify a folder for disk offloading
)

model.load_adapter(ADAPTER_PATH)
model.set_adapter("default")

print("SRKR Expert ready! 🚀")


Using offload folder: /tmp/tmpwli8bbfo


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

SRKR Expert ready! 🚀


In [ ]:
def ask_model(question):
    model.eval()
    prompt = f"### Question: {question}\n### Answer:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.3,  # stable answers
            top_p=0.9,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove prompt part from output
    response = text.replace(prompt, "").strip()

    # stop before next question (if the model generates one)
    response = response.split("### Question:")[0].strip()

    return response


print("SRKR Expert Model Ready! 🚀")
print("Answer:", ask_model("name the faculty of cloud computing ?"))


SRKR Expert Model Ready! 🚀
Answer: The faculty specialized in Cloud Computing are: Chintapalli Siva Subrahmanyam, B. Teja Sree, N. Deshai, G. Sandeep, Dr. S. Venkataramana.


In [1]:
!git config --global user.email "harshavardhantamada333@gmail.com"
!git config --global user.name "harshatamada"


In [2]:
!git clone https://github.com/harshatamada/SystemLabs

Cloning into 'SystemLabs'...


In [6]:
!ls "/content/drive/MyDrive/system_labs"




adapters
